In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE  # Để xử lý imbalanced data
import joblib
import sys
import os
import importlib

# Thêm thư mục cha của notebooks vào sys.path
sys.path.append(os.path.abspath('..'))

# Import và reload để lấy phiên bản mới nhất
from utils import embedding
importlib.reload(embedding)
from utils.embedding import get_phobert_embeddings_batch

print("✓ Đã import tất cả thư viện")

✓ Đã import tất cả thư viện


In [2]:
# Load dữ liệu đã xử lý
df = pd.read_csv('../data/processed/cleaned_reviews.csv')

# Load PhoBERT và chuyển sang GPU nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Sử dụng device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert.to(device)
print("✓ Đã load PhoBERT model")

🖥️  Sử dụng device: cuda
   GPU: NVIDIA GeForce RTX 4050 Laptop GPU
   Memory: 6.05 GB
✓ Đã load PhoBERT model


In [3]:
# Tạo embeddings với batch processing (tận dụng GPU hiệu quả hơn)
# Loại bỏ các dòng có review_cleaned rỗng hoặc NaN
df_clean = df.dropna(subset=['review_cleaned'])
df_clean = df_clean[df_clean['review_cleaned'].str.strip() != ''].reset_index(drop=True)

print(f"📝 Tổng số mẫu hợp lệ: {len(df_clean)} (đã loại bỏ {len(df) - len(df_clean)} mẫu không hợp lệ)")
print(f"🔄 Đang tạo embeddings trên {device}...")

# Sử dụng batch processing để tận dụng GPU
batch_size = 64 if torch.cuda.is_available() else 32
embeddings = get_phobert_embeddings_batch(
    df_clean['review_cleaned'].tolist(), 
    phobert, 
    tokenizer, 
    device,
    batch_size=batch_size
)

np.save('../data/processed/embeddings.npy', embeddings)
np.save('../data/processed/labels.npy', df_clean['label'].values)
print(f"✓ Đã tạo embeddings với shape: {embeddings.shape}")
print(f"✓ Lưu embeddings vào '../data/processed/embeddings.npy'")
print(f"✓ Lưu labels vào '../data/processed/labels.npy'")

📝 Tổng số mẫu hợp lệ: 22822 (đã loại bỏ 21 mẫu không hợp lệ)
🔄 Đang tạo embeddings trên cuda...


Tạo embeddings:   0%|          | 0/357 [00:00<?, ?it/s]

Tạo embeddings: 100%|██████████| 357/357 [03:38<00:00,  1.63it/s]

✓ Đã tạo embeddings với shape: (22822, 768)
✓ Lưu embeddings vào '../data/processed/embeddings.npy'
✓ Lưu labels vào '../data/processed/labels.npy'


In [4]:
# Chia train/validation/test (60/20/20) để theo dõi overfitting
X_temp, X_test, y_temp, y_test = train_test_split(
    embeddings, df_clean['label'].values, test_size=0.2, random_state=42, stratify=df_clean['label']
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp  # 0.25 * 0.8 = 0.2
)

print(f"📊 Chia dữ liệu:")
print(f"- Train: {len(X_train)} mẫu")
print(f"- Validation: {len(X_val)} mẫu")
print(f"- Test: {len(X_test)} mẫu")
print(f"\n⚠️  Phân bố nhãn trong train:")
print(f"- Negative: {(y_train == 0).sum()} ({100*(y_train == 0).sum()/len(y_train):.1f}%)")
print(f"- Positive: {(y_train == 1).sum()} ({100*(y_train == 1).sum()/len(y_train):.1f}%)")

# Huấn luyện các model với class balancing và regularization mạnh
print("\n🤖 Bắt đầu huấn luyện các mô hình (với class_weight='balanced')...\n")
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000, 
        C=0.1,  # Regularization mạnh hơn (giảm từ 1.0 xuống 0.1)
        class_weight='balanced',  # Xử lý imbalanced data
        random_state=42
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=50,  # Giảm từ 100 xuống 50
        max_depth=10,  # Giới hạn độ sâu để giảm overfitting
        min_samples_split=20,  # Tăng từ 2 lên 20
        min_samples_leaf=10,  # Tăng từ 1 lên 10
        class_weight='balanced',
        random_state=42
    ),
    'SVM': SVC(
        kernel='rbf',
        C=0.5,  # Regularization mạnh hơn (giảm từ 1.0 xuống 0.5)
        gamma='scale',
        class_weight='balanced',
        random_state=42
    )
}

results = {}
results_val = {}
for name, model in models.items():
    print(f"⏳ Đang huấn luyện {name}...")
    model.fit(X_train, y_train)
    
    # Đánh giá trên cả train, validation và test
    train_acc = accuracy_score(y_train, model.predict(X_train))
    val_acc = accuracy_score(y_val, model.predict(X_val))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    
    results[name] = test_acc
    results_val[name] = val_acc
    
    print(f"✅ {name}:")
    print(f"   - Train: {100 * train_acc:.2f}%")
    print(f"   - Val:   {100 * val_acc:.2f}%")
    print(f"   - Test:  {100 * test_acc:.2f}%")
    print(f"   - Overfitting gap: {100 * (train_acc - val_acc):.2f}%\n")

print("=" * 60)
print("📈 KẾT QUẢ HUẤN LUYỆN TẤT CẢ CÁC MÔ HÌNH (sắp xếp theo Val):")
print("=" * 60)
for name in sorted(results_val, key=results_val.get, reverse=True):
    print(f"{name:20s}: Val={100*results_val[name]:.2f}% | Test={100*results[name]:.2f}%")

📊 Chia dữ liệu:
- Train: 13692 mẫu
- Validation: 4565 mẫu
- Test: 4565 mẫu

⚠️  Phân bố nhãn trong train:
- Negative: 3291 (24.0%)
- Positive: 10401 (76.0%)

🤖 Bắt đầu huấn luyện các mô hình (với class_weight='balanced')...

⏳ Đang huấn luyện Logistic Regression...
✅ Logistic Regression:
   - Train: 89.29%
   - Val:   88.54%
   - Test:  88.50%
   - Overfitting gap: 0.75%

⏳ Đang huấn luyện Random Forest...
✅ Random Forest:
   - Train: 97.63%
   - Val:   91.48%
   - Test:  91.37%
   - Overfitting gap: 6.15%

⏳ Đang huấn luyện SVM...
✅ SVM:
   - Train: 91.27%
   - Val:   90.21%
   - Test:  89.86%
   - Overfitting gap: 1.06%

📈 KẾT QUẢ HUẤN LUYỆN TẤT CẢ CÁC MÔ HÌNH (sắp xếp theo Val):
Random Forest       : Val=91.48% | Test=91.37%
SVM                 : Val=90.21% | Test=89.86%
Logistic Regression : Val=88.54% | Test=88.50%


In [8]:
# ============================================================
# ⚡ CẢI TIẾN: Áp dụng SMOTE để cân bằng dữ liệu training
# ============================================================
print("\n🔄 Áp dụng SMOTE để cân bằng dữ liệu training...")
print(f"Trước SMOTE:")
print(f"  - Negative: {(y_train == 0).sum()}")
print(f"  - Positive: {(y_train == 1).sum()}")
print(f"  - Tỷ lệ: {(y_train == 1).sum() / (y_train == 0).sum():.2f}:1")

# Áp dụng SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\nSau SMOTE:")
print(f"  - Negative: {(y_train_balanced == 0).sum()}")
print(f"  - Positive: {(y_train_balanced == 1).sum()}")
print(f"  - Tỷ lệ: {(y_train_balanced == 1).sum() / (y_train_balanced == 0).sum():.2f}:1")
print(f"\n✅ Đã tạo thêm {len(X_train_balanced) - len(X_train)} mẫu synthetic\n")


🔄 Áp dụng SMOTE để cân bằng dữ liệu training...
Trước SMOTE:
  - Negative: 3291
  - Positive: 10401
  - Tỷ lệ: 3.16:1

Sau SMOTE:
  - Negative: 10401
  - Positive: 10401
  - Tỷ lệ: 1.00:1

✅ Đã tạo thêm 7110 mẫu synthetic



In [9]:
# ============================================================
# 🤖 TRAIN LẠI VỚI DỮ LIỆU ĐÃ CÂN BẰNG
# ============================================================
print("🚀 Huấn luyện lại với dữ liệu đã cân bằng bởi SMOTE...\n")

# Định nghĩa models với regularization CỰC MẠNH
models_balanced = {
    'Logistic Regression (SMOTE)': LogisticRegression(
        max_iter=2000,
        C=0.01,  # Regularization CỰC MẠNH (giảm từ 0.1 xuống 0.01)
        penalty='l2',
        solver='lbfgs',
        random_state=42,
        # Không dùng class_weight nữa vì đã SMOTE
    ),
    'Random Forest (SMOTE)': RandomForestClassifier(
        n_estimators=30,  # Giảm xuống 30
        max_depth=8,  # Giảm xuống 8
        min_samples_split=30,  # Tăng lên 30
        min_samples_leaf=15,  # Tăng lên 15
        max_features='sqrt',  # Giới hạn features
        random_state=42,
    ),
    'SVM (SMOTE)': SVC(
        kernel='rbf',
        C=0.1,  # Regularization mạnh hơn
        gamma='scale',
        random_state=42,
        probability=True  # Để có thể dùng predict_proba
    )
}

results_balanced = {}
results_val_balanced = {}

for name, model in models_balanced.items():
    print(f"⏳ Đang huấn luyện {name}...")
    
    # Train trên dữ liệu đã balanced
    model.fit(X_train_balanced, y_train_balanced)
    
    # Đánh giá trên tất cả các tập
    train_acc = accuracy_score(y_train_balanced, model.predict(X_train_balanced))
    val_acc = accuracy_score(y_val, model.predict(X_val))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    
    # Dự đoán trên validation để kiểm tra phân bố
    y_val_pred_temp = model.predict(X_val)
    neg_pred = (y_val_pred_temp == 0).sum()
    pos_pred = (y_val_pred_temp == 1).sum()
    
    results_balanced[name] = test_acc
    results_val_balanced[name] = val_acc
    
    print(f"✅ {name}:")
    print(f"   - Train: {100 * train_acc:.2f}%")
    print(f"   - Val:   {100 * val_acc:.2f}%")
    print(f"   - Test:  {100 * test_acc:.2f}%")
    print(f"   - Overfitting gap: {100 * (train_acc - val_acc):.2f}%")
    print(f"   - Dự đoán Val: Neg={neg_pred} ({100*neg_pred/len(y_val):.1f}%), Pos={pos_pred} ({100*pos_pred/len(y_val):.1f}%)")
    print()

print("=" * 70)
print("📊 SO SÁNH KẾT QUẢ: CLASS_WEIGHT vs SMOTE")
print("=" * 70)
print("\n🔵 Với class_weight='balanced':")
for name in sorted(results_val, key=results_val.get, reverse=True):
    print(f"  {name:25s}: Val={100*results_val[name]:.2f}% | Test={100*results[name]:.2f}%")

print("\n🟢 Với SMOTE:")
for name in sorted(results_val_balanced, key=results_val_balanced.get, reverse=True):
    print(f"  {name:25s}: Val={100*results_val_balanced[name]:.2f}% | Test={100*results_balanced[name]:.2f}%")
print("=" * 70)

🚀 Huấn luyện lại với dữ liệu đã cân bằng bởi SMOTE...

⏳ Đang huấn luyện Logistic Regression (SMOTE)...


/home/lucchuong/Documents/sentiment-classification/training/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


🚀 Huấn luyện lại với dữ liệu đã cân bằng bởi SMOTE...

⏳ Đang huấn luyện Logistic Regression (SMOTE)...


/home/lucchuong/Documents/sentiment-classification/training/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


✅ Logistic Regression (SMOTE):
   - Train: 88.68%
   - Val:   88.52%
   - Test:  88.37%
   - Overfitting gap: 0.16%
   - Dự đoán Val: Neg=1329 (29.1%), Pos=3236 (70.9%)

⏳ Đang huấn luyện Random Forest (SMOTE)...
✅ Random Forest (SMOTE):
   - Train: 94.03%
   - Val:   88.54%
   - Test:  88.17%
   - Overfitting gap: 5.49%
   - Dự đoán Val: Neg=1244 (27.3%), Pos=3321 (72.7%)

⏳ Đang huấn luyện SVM (SMOTE)...
✅ SVM (SMOTE):
   - Train: 90.14%
   - Val:   89.57%
   - Test:  89.24%
   - Overfitting gap: 0.57%
   - Dự đoán Val: Neg=1193 (26.1%), Pos=3372 (73.9%)

📊 SO SÁNH KẾT QUẢ: CLASS_WEIGHT vs SMOTE

🔵 Với class_weight='balanced':
  Random Forest            : Val=91.48% | Test=91.37%
  SVM                      : Val=90.21% | Test=89.86%
  Logistic Regression      : Val=88.54% | Test=88.50%

🟢 Với SMOTE:
  SVM (SMOTE)              : Val=89.57% | Test=89.24%
  Random Forest (SMOTE)    : Val=88.54% | Test=88.17%
  Logistic Regression (SMOTE): Val=88.52% | Test=88.37%


In [10]:
# ============================================================
# 🏆 CHỌN VÀ LƯU MODEL TỐT NHẤT TỪ SMOTE
# ============================================================
# Chọn model tốt nhất dựa trên validation accuracy từ SMOTE
best_model_name_new = max(results_val_balanced, key=results_val_balanced.get)
best_model_new = models_balanced[best_model_name_new]

# Lưu model mới
os.makedirs('../models', exist_ok=True)
joblib.dump(best_model_new, '../models/best_sentiment_model.pkl')

print("\n" + "=" * 70)
print("🏆 MODEL TỐT NHẤT (SMOTE):", best_model_name_new)
print("=" * 70)
print(f"🎯 Validation Accuracy: {100 * results_val_balanced[best_model_name_new]:.2f}%")
print(f"🎯 Test Accuracy: {100 * results_balanced[best_model_name_new]:.2f}%")
print(f"✓ Đã lưu model vào '../models/best_sentiment_model.pkl'")

# Phân tích chi tiết trên validation set
print("\n📊 PHÂN TÍCH CHI TIẾT TRÊN VALIDATION SET:")
y_val_pred_final = best_model_new.predict(X_val)

from collections import Counter
print(f"\nPhân bố DỰ ĐOÁN: {Counter(y_val_pred_final)}")
print(f"Phân bố THỰC TẾ: {Counter(y_val)}")

print(f"\n- Negative:")
print(f"  Dự đoán: {(y_val_pred_final == 0).sum()} ({100*(y_val_pred_final == 0).sum()/len(y_val):.1f}%)")
print(f"  Thực tế:  {(y_val == 0).sum()} ({100*(y_val == 0).sum()/len(y_val):.1f}%)")

print(f"\n- Positive:")
print(f"  Dự đoán: {(y_val_pred_final == 1).sum()} ({100*(y_val_pred_final == 1).sum()/len(y_val):.1f}%)")
print(f"  Thực tế:  {(y_val == 1).sum()} ({100*(y_val == 1).sum()/len(y_val):.1f}%)")

# Classification report
print("\n📋 CLASSIFICATION REPORT (Validation Set):")
print(classification_report(y_val, y_val_pred_final, 
                          target_names=['Negative', 'Positive'],
                          digits=3))

# Confusion Matrix
print("\n📊 CONFUSION MATRIX (Validation Set):")
cm = confusion_matrix(y_val, y_val_pred_final)
print(f"                Predicted")
print(f"              Neg    Pos")
print(f"Actual Neg   {cm[0][0]:4d}   {cm[0][1]:4d}")
print(f"       Pos   {cm[1][0]:4d}   {cm[1][1]:4d}")
print("=" * 70)


🏆 MODEL TỐT NHẤT (SMOTE): SVM (SMOTE)
🎯 Validation Accuracy: 89.57%
🎯 Test Accuracy: 89.24%
✓ Đã lưu model vào '../models/best_sentiment_model.pkl'

📊 PHÂN TÍCH CHI TIẾT TRÊN VALIDATION SET:

Phân bố DỰ ĐOÁN: Counter({np.int64(1): 3372, np.int64(0): 1193})
Phân bố THỰC TẾ: Counter({np.int64(1): 3468, np.int64(0): 1097})

- Negative:
  Dự đoán: 1193 (26.1%)
  Thực tế:  1097 (24.0%)

- Positive:
  Dự đoán: 3372 (73.9%)
  Thực tế:  3468 (76.0%)

📋 CLASSIFICATION REPORT (Validation Set):
              precision    recall  f1-score   support

    Negative      0.760     0.827     0.792      1097
    Positive      0.944     0.918     0.930      3468

    accuracy                          0.896      4565
   macro avg      0.852     0.872     0.861      4565
weighted avg      0.900     0.896     0.897      4565


📊 CONFUSION MATRIX (Validation Set):
                Predicted
              Neg    Pos
Actual Neg    907    190
       Pos    286   3182


In [5]:
# Chọn mô hình tốt nhất dựa trên validation accuracy
best_model_name = max(results_val, key=results_val.get)
best_model = models[best_model_name]

# Tạo thư mục models nếu chưa có
os.makedirs('../models', exist_ok=True)

joblib.dump(best_model, '../models/best_sentiment_model.pkl')
print(f"\n🏆 MÔ HÌNH TỐT NHẤT: {best_model_name}")
print(f"🎯 Validation Accuracy: {100 * results_val[best_model_name]:.2f}%")
print(f"🎯 Test Accuracy: {100 * results[best_model_name]:.2f}%")
print(f"✓ Đã lưu mô hình vào '../models/best_sentiment_model.pkl'")

# Kiểm tra dự đoán trên validation set
print("\n📊 Phân tích dự đoán trên Validation Set:")
y_val_pred = best_model.predict(X_val)
from collections import Counter
print(f"Phân bố dự đoán: {Counter(y_val_pred)}")
print(f"Phân bố thực tế: {Counter(y_val)}")
print(f"- Negative (0): {(y_val_pred == 0).sum()} dự đoán vs {(y_val == 0).sum()} thực tế")
print(f"- Positive (1): {(y_val_pred == 1).sum()} dự đoán vs {(y_val == 1).sum()} thực tế")


🏆 MÔ HÌNH TỐT NHẤT: Random Forest
🎯 Validation Accuracy: 91.48%
🎯 Test Accuracy: 91.37%
✓ Đã lưu mô hình vào '../models/best_sentiment_model.pkl'

📊 Phân tích dự đoán trên Validation Set:
Phân bố dự đoán: Counter({np.int64(1): 3573, np.int64(0): 992})
Phân bố thực tế: Counter({np.int64(1): 3468, np.int64(0): 1097})
- Negative (0): 992 dự đoán vs 1097 thực tế
- Positive (1): 3573 dự đoán vs 3468 thực tế
